In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from layout import Layout

In [2]:
countries = pd.read_csv("mig_nodes.csv", index_col=0)
migration = pd.read_csv("mig_edges.csv")

In [3]:
countries

,name,code,region,sub_region,pos
index,,,,,
0,Afghanistan,4,Asia,Southern Asia,"array([1.83174148, 1.86242262])"
1,Albania,8,Europe,Southern Europe,"array([1.89105956, 2.05459683])"
2,Algeria,12,Africa,Northern Africa,"array([1.90130528, 1.98917661])"
3,American Samoa,16,Oceania,Polynesia,"array([1.97167436, 2.13972604])"
4,Andorra,20,Europe,Southern Europe,"array([1.70167317, 1.89743272])"
...,...,...,...,...,...
227,Wallis and Futuna Islands,876,Oceania,Polynesia,"array([1.79672158, 2.22175723])"
228,Western Sahara,732,Africa,Northern Africa,"array([2.01706011, 1.86488782])"
229,Yemen,887,Asia,Western Asia,"array([1.92891113, 1.97069976])"


In [4]:
migration["total"] = migration[["migration_1990_total","migration_1995_total","migration_2000_total","migration_2005_total","migration_2010_total","migration_2015_total"]].sum(axis=1)
migration = migration[migration["total"] != 0]

In [5]:
migration

,source,target,migration_1990_total,migration_1990_male,migration_1990_female,migration_1995_total,migration_1995_male,migration_1995_female,migration_2000_total,migration_2000_male,...,migration_2005_total,migration_2005_male,migration_2005_female,migration_2010_total,migration_2010_male,migration_2010_female,migration_2015_total,migration_2015_male,migration_2015_female,total
0,0,64,237,194,43,188,150,38,154,117,...,183,142,41,198,152,46,235,173,62,1195
1,0,118,677,556,121,452,351,101,236,155,...,259,187,72,284,206,78,320,231,89,2228
2,0,142,64,26,38,61,25,36,58,23,...,45,18,27,43,18,25,39,17,22,310
3,0,191,59,37,22,36,24,12,30,20,...,47,27,20,69,41,28,83,50,33,324
4,0,202,8485,3726,4759,12248,5518,6730,17118,7874,...,12636,5687,6949,8900,3906,4994,7587,3279,4308,66974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11223,231,220,4767,2148,2619,7571,3451,4120,10686,4909,...,26585,12282,14303,42811,19381,23430,50001,22556,27445,142421
11224,231,11,8265,4184,4081,10498,5291,5207,12110,6110,...,21690,10960,10730,33720,16870,16850,38843,19418,19425,125126
11225,231,147,676,323,353,822,398,424,2807,1365,...,7794,3783,4011,8170,3959,4211,8416,4070,4346,28685
11226,231,68,0,0,0,0,0,0,0,0,...,0,0,0,6,6,0,6,6,0,12


In [6]:
G = nx.DiGraph()
for _, row in migration.iterrows():
    G.add_edge(row["source"], row["target"], weight=row["total"])
nx.info(G)

'DiGraph with 232 nodes and 11159 edges'

In [7]:
pos = nx.spring_layout(G, dim=3)

In [8]:
layout = Layout(G, pos)

In [9]:
layout.node_names = countries["name"]

In [10]:
green = (56, 142, 60, 100)
red = (211, 47, 47, 100)
grey = (97, 97, 97, 100)

def get_node_color(G, node):
    in_deg = G.in_degree(node, weight="weight")
    out_deg = G.out_degree(node, weight="weight")
    net = in_deg - out_deg
    if net > 0:
        return green
    if net < 0:
        return red
    return grey

layout.node_colors = [get_node_color(G, node) for node in G]

In [11]:
def get_edge_color(weight):
    glow = 100 + 155 * np.log10(weight) / 7
    return (255, 255, 255, int(glow))

layout.edge_colors = [get_edge_color(weight) for weight in migration["total"]]

In [12]:
layout.node_table

,x,y,z,r,g,b,a,name
0,0.755213,0.665079,0.481842,211,47,47,100,Afghanistan
1,0.556768,0.597673,0.562465,211,47,47,100,Albania
2,0.562944,0.650924,0.556544,56,142,60,100,Algeria
3,0.656600,0.583454,0.561743,56,142,60,100,American Samoa
4,0.661652,0.617225,0.534849,56,142,60,100,Andorra
...,...,...,...,...,...,...,...,...
227,0.000000,0.588475,0.786583,211,47,47,100,Wallis and Futuna Islands
228,0.678553,0.634400,0.511474,56,142,60,100,Western Sahara
229,0.546101,0.577612,0.589633,211,47,47,100,Yemen
230,0.437229,0.000000,0.306830,211,47,47,100,Zambia


In [13]:
layout.edge_table

,i,j,r,g,b,a
0,0,1,255,255,255,168
1,0,2,255,255,255,174
2,0,3,255,255,255,155
3,0,4,255,255,255,155
4,0,5,255,255,255,206
...,...,...,...,...,...,...
11154,231,113,255,255,255,214
11155,231,58,255,255,255,212
11156,231,137,255,255,255,198
11157,231,61,255,255,255,123


In [14]:
layout.write(node_file_path="migration_dir_nodes.csv", edge_file_path="migration_dir_edges.csv")

### Some network attributes

In [15]:
# density
nx.density(G)

0.2082213763248246

In [16]:
# average clustering coefficient
nx.average_clustering(G, weight="weight")

0.00021035824517491128

In [17]:
# average next nearest neighbor weight
degs = nx.average_degree_connectivity(G, source='in+out', target='in+out', weight="weight")
np.mean(list(degs.values()))

120.56964613811137

In [18]:
# number of detected communities
partitions = nx.algorithms.community.greedy_modularity_communities(G, weight="weight")
len(partitions)

8

In [19]:
# assortativity
nx.degree_pearson_correlation_coefficient(G, x='out', y='in', weight="weight")

-0.0356085424256383